In [2]:
import sqlite3

# Connect to SQLite database (or create one if it doesn't exist)
conn = sqlite3.connect("ecommerce.db")
cursor = conn.cursor()

# Create tables (Customers, Orders, Products)
cursor.execute("""
    CREATE TABLE IF NOT EXISTS Customers (
        CustomerID INTEGER PRIMARY KEY,
        Name TEXT,
        Email TEXT
    );
""")

cursor.execute("""
    CREATE TABLE IF NOT EXISTS Orders (
        OrderID INTEGER PRIMARY KEY,
        CustomerID INTEGER,
        OrderDate DATE,
        TotalAmount REAL,
        FOREIGN KEY(CustomerID) REFERENCES Customers(CustomerID)
    );
""")

cursor.execute("""
    CREATE TABLE IF NOT EXISTS Products (
        ProductID INTEGER PRIMARY KEY,
        Name TEXT,
        Price REAL,
        Category TEXT
    );
""")

In [3]:
# Insert sample data into tables
cursor.executemany("INSERT INTO Customers (Name, Email) VALUES (?, ?)", [
    ("Alice", "alice@example.com"),
    ("Bob", "bob@example.com"),
    ("Charlie", "charlie@example.com")
])

cursor.executemany("INSERT INTO Orders (CustomerID, OrderDate, TotalAmount) VALUES (?, ?, ?)", [
    (1, "2025-02-10", 250.50),
    (2, "2025-03-15", 120.75),
    (3, "2025-01-05", 600.30)
])

cursor.executemany("INSERT INTO Products (Name, Price, Category) VALUES (?, ?, ?)", [
    ("Laptop", 1000.00, "Electronics"),
    ("Phone", 700.00, "Electronics"),
    ("Shoes", 100.00, "Fashion"),
    ("T-shirt", 30.00, "Fashion")
])

In [4]:
# Select all customers
cursor.execute("SELECT * FROM Customers;")
customers = cursor.fetchall()
print("Customers:", customers)

# Orders after a certain date
cursor.execute("SELECT * FROM Orders WHERE OrderDate >= '2025-01-01';")
orders = cursor.fetchall()
print("Recent Orders:", orders)

# Products sorted by price (descending)
cursor.execute("SELECT * FROM Products ORDER BY Price DESC;")
products = cursor.fetchall()
print("Products sorted by price:", products)

# Grouping products by category
cursor.execute("SELECT Category, COUNT(*) FROM Products GROUP BY Category;")
categories = cursor.fetchall()
print("Products per category:", categories)

Customers: [(1, 'Alice', 'alice@example.com'), (2, 'Bob', 'bob@example.com'), (3, 'Charlie', 'charlie@example.com')]
Recent Orders: [(1, 1, '2025-02-10', 250.5), (2, 2, '2025-03-15', 120.75), (3, 3, '2025-01-05', 600.3)]
Products sorted by price: [(1, 'Laptop', 1000.0, 'Electronics'), (2, 'Phone', 700.0, 'Electronics'), (3, 'Shoes', 100.0, 'Fashion'), (4, 'T-shirt', 30.0, 'Fashion')]
Products per category: [('Electronics', 2), ('Fashion', 2)]


In [5]:
# INNER JOIN Customers & Orders
cursor.execute("""
    SELECT Customers.Name, Orders.OrderID 
    FROM Customers 
    INNER JOIN Orders ON Customers.CustomerID = Orders.CustomerID;
""")
joined_data = cursor.fetchall()
print("Customer Orders:", joined_data)

# Most expensive product using subquery
cursor.execute("SELECT Name, Price FROM Products WHERE Price = (SELECT MAX(Price) FROM Products);")
expensive_product = cursor.fetchone()
print("Most expensive product:", expensive_product)

# Total revenue & average order value
cursor.execute("SELECT SUM(TotalAmount) FROM Orders;")
total_revenue = cursor.fetchone()
print("Total Revenue:", total_revenue)

cursor.execute("SELECT AVG(TotalAmount) FROM Orders;")
avg_order_value = cursor.fetchone()
print("Average Order Value:", avg_order_value)

Customer Orders: [('Alice', 1), ('Bob', 2), ('Charlie', 3)]
Most expensive product: ('Laptop', 1000.0)
Total Revenue: (971.55,)
Average Order Value: (323.84999999999997,)


In [6]:
# Create a view for customer orders
cursor.execute("""
    CREATE VIEW IF NOT EXISTS CustomerOrders AS 
    SELECT Customers.Name, Orders.OrderDate, Orders.TotalAmount 
    FROM Customers 
    INNER JOIN Orders ON Customers.CustomerID = Orders.CustomerID;
""")

# Query the view for high-value orders
cursor.execute("SELECT * FROM CustomerOrders WHERE TotalAmount > 500;")
high_value_orders = cursor.fetchall()
print("High Value Orders:", high_value_orders)

# Indexing for optimization
cursor.execute("CREATE INDEX IF NOT EXISTS idx_CustomerID ON Orders (CustomerID);")
cursor.execute("CREATE INDEX IF NOT EXISTS idx_Category ON Products (Category);")

High Value Orders: [('Charlie', '2025-01-05', 600.3)]


In [9]:
# Commit changes and close connection
#conn.commit()
#conn.close()